In [5]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
train_dir = '/content/gdrive/MyDrive/train'
normal_dir = '/content/gdrive/MyDrive/train/NORMAL'
pneumonia_dir = '/content/gdrive/MyDrive/train/PNEUMONIA'

In [10]:
img_size = (150, 150)
batch_size = 32

# Use data augmentation to increase training data size
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Load training data from directories
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='binary')

Found 5216 images belonging to 2 classes.


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator,
                    epochs=20)


Epoch 1/20
163/163 [==============================] - 92s 550ms/step - loss: 0.1318 - accuracy: 0.9490
Epoch 2/20
163/163 [==============================] - 88s 540ms/step - loss: 0.1370 - accuracy: 0.9509
Epoch 3/20
163/163 [==============================] - 88s 543ms/step - loss: 0.1277 - accuracy: 0.9534
Epoch 4/20
163/163 [==============================] - 88s 539ms/step - loss: 0.1228 - accuracy: 0.9578
Epoch 5/20
163/163 [==============================] - 89s 548ms/step - loss: 0.1210 - accuracy: 0.9571
Epoch 6/20
163/163 [==============================] - 88s 539ms/step - loss: 0.1193 - accuracy: 0.9555
Epoch 7/20
163/163 [==============================] - 89s 546ms/step - loss: 0.1091 - accuracy: 0.9588
Epoch 8/20
163/163 [==============================] - 88s 541ms/step - loss: 0.1099 - accuracy: 0.9592
Epoch 9/20
163/163 [==============================] - 89s 547ms/step - loss: 0.1024 - accuracy: 0.9613
Epoch 10/20
163/163 [==============================] - 88s 539ms/step - l

In [27]:
model.save('model.h5')

In [28]:
import cv2
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model


test_dir = '/content/gdrive/MyDrive/test'
test_df = pd.DataFrame(columns=['id', 'labels'])

model = load_model('model.h5')

In [29]:
for i, file_name in enumerate(os.listdir(test_dir)):
    img_path = os.path.join(test_dir, file_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150, 150))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    pred = model.predict(img)[0][0]
    if pred > 0.5:
        label = 1  # Pneumonia
    else:
        label = 0  # Normal
    test_df.loc[i] = [file_name, label]

1/1 [==============================] - 0s 24ms/step


In [ ]:
print(test_df)

In [33]:
test_df.to_csv('test_results1.csv', index=False)